In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import onnx
import sys
import re
import torch.optim as optim
import time
from torch.utils.data import DataLoader
from scipy import ndimage
from tqdm import tqdm
from datetime import datetime

In [2]:
sys.path.append('..')
from classes.dataset_utils.toTorchDataset import ProcessedKit23TorchDataset
from classes.models import resnet_model_generator
from classes.config_class import ProjectModelResnetConfig
from classes.epoch_results import EpochResult

In [3]:
training_data = ProcessedKit23TorchDataset(train_data=True, test_size=0.25, dataset_dir ="./dataset/affine_transformed")
test_data = ProcessedKit23TorchDataset(train_data=False, test_size=0.25, dataset_dir ="./dataset/affine_transformed")

In [4]:
proj_config = ProjectModelResnetConfig(model_depth=50)
proj_resnet_model, _ = resnet_model_generator.generate_model(proj_config)

In [5]:
proj_config.set_net_model(proj_resnet_model)

In [6]:
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(proj_config.nn_model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
if not proj_config.no_cuda:
    criterion = criterion.cuda()

In [7]:
train_from_pretrained = False
epoch_res = EpochResult()
epoch_start = 0
if train_from_pretrained:
    print("loading from pretrained Med3D model")
    if proj_config.model_depth == 10:
        proj_config.load_med3d_pretrain_weigth("../pretrainedModel/resnet_10_23dataset.pth")
    elif proj_config.model_depth == 50:
        proj_config.load_med3d_pretrain_weigth("../pretrainedModel/resnet_50_23dataset.pth")
    else:
        raise Exception("Only depth 10 and 50 are used for now.")
else:
    # this continues from certain training points
    checkpoint, epoch_res = proj_config.load_weight_from_epoch("./training_checkpoints/Model_resnet_50_epoch5.pth.tar")
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = epoch_res.epoch_list[-1] + 1
    
 

./training_checkpoints/Model_resnet_50_epoch5.pth.tar


In [8]:
data_loader = DataLoader(training_data, batch_size=proj_config.batch_size, shuffle=True, num_workers=proj_config.num_workers, pin_memory=proj_config.pin_memory)

In [9]:
train_time_start = time.time()
batches_per_epoch = len(data_loader)

for epoch in range(epoch_start, proj_config.max_epoch):
    current_lr = scheduler.get_last_lr()
    running_loss = None
    print("current epoch={:5d} Learning Rate={}".format(epoch, current_lr))
    
    for batch_idx, batch_data  in enumerate(data_loader):
        imgs, segs = batch_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        y_preds = proj_config.nn_model(imgs.float())

        [n, _, z_size, y_size, x_size] = y_preds.shape

        resized_segs = np.zeros([n, z_size, y_size, x_size])
        for idx in range(n):
            seg = segs[idx][0]
            [ori_z, ori_y, ori_x] = seg.shape 
            scale = [z_size/ori_z, y_size/ori_y, x_size/ori_x]
            this_affine = np.array([[scale[0], 0, 0],[0, scale[1], 0],[0, 0, scale[2]]])
            resized_segs[idx] = ndimage.affine_transform(seg, this_affine, output_shape=resized_segs[idx].shape, cval=0)

        resized_segs = torch.tensor(resized_segs).to(torch.int64)
        loss = criterion(y_preds, resized_segs)
        running_loss = loss.item()
        loss.backward()                
        optimizer.step()
        
        
        total_processed_batches = (epoch - epoch_start) * batches_per_epoch + 1 + batch_idx
        avg_batch_time = (time.time() - train_time_start) / total_processed_batches
        if batch_idx % 50 == 0:
            print("Epoch:{} Batch:{} loss = {:.5f}, avg_batch_time = {:.5f}".format(epoch, batch_idx, running_loss, avg_batch_time))
    scheduler.step()
    epoch_res.append_result(epoch, running_loss, current_lr)
    model_checkpoint_path = proj_config.save_checkpoint_pathname(epoch, with_Datetime=False)
    torch.save({'epoch_list': epoch_res.epoch_list, 'loss_list': epoch_res.loss_list, 'lr_list': epoch_res.lr_list, 
                'state_dict': proj_config.nn_model.state_dict(),'optimizer': optimizer.state_dict()},model_checkpoint_path, _use_new_zipfile_serialization=True)

print('Finished Training')

current epoch=    6 Learning Rate=[0.001]
Epoch:6 Batch:0 loss = 0.00232, avg_batch_time = 29.98519
Epoch:6 Batch:50 loss = 0.00231, avg_batch_time = 23.71196
Epoch:6 Batch:100 loss = 0.00224, avg_batch_time = 21.16570
Epoch:6 Batch:150 loss = 0.00220, avg_batch_time = 19.81156
Epoch:6 Batch:200 loss = 0.00218, avg_batch_time = 18.98279
Epoch:6 Batch:250 loss = 0.00213, avg_batch_time = 18.47534
Epoch:6 Batch:300 loss = 0.00209, avg_batch_time = 18.13857
Epoch:6 Batch:350 loss = 0.00204, avg_batch_time = 20.75054
current epoch=    7 Learning Rate=[0.0009320653479069899]
Epoch:7 Batch:0 loss = 0.00205, avg_batch_time = 20.48641
Epoch:7 Batch:50 loss = 0.00204, avg_batch_time = 19.97172
Epoch:7 Batch:100 loss = 0.00199, avg_batch_time = 19.57779
Epoch:7 Batch:150 loss = 0.00197, avg_batch_time = 19.24084
Epoch:7 Batch:200 loss = 0.00192, avg_batch_time = 18.96463
Epoch:7 Batch:250 loss = 0.00195, avg_batch_time = 18.73003
Epoch:7 Batch:300 loss = 0.00200, avg_batch_time = 18.53045
Epoch:

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x112902290>
Traceback (most recent call last):
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/local/opt/pyenv/versions/3.10.9/lib/python3.10/selectors.py", line 416, in

KeyboardInterrupt: 